In [33]:
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [34]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset2/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset2/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset2/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset2/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [35]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)


In [36]:

x_train.shape

(35471, 6, 128)

In [37]:
embed_dim = x_train.shape[-1]   # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = x_train.shape[-1]  # Hidden layer size in feed forward network inside transformer

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm
model = keras.Sequential()
model.add(layers.Input(shape= x_train.shape[1:]))
model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(ff_dim, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(20, activation='softmax'))

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer_block_4 (Transf  (None, 6, 128)           297344    
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_4   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 128)               16512     
                                                                 
 dropout_19 (Dropout)        (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 20)               

In [38]:
history = model.fit(x_train, y_train, epochs=200, verbose=True, validation_data=(x_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/200
278/278 [==============================] - 6s 16ms/step - loss: 1.9576 - accuracy: 0.4405 - val_loss: 1.0136 - val_accuracy: 0.7216
Epoch 2/200
278/278 [==============================] - 4s 15ms/step - loss: 0.9239 - accuracy: 0.7488 - val_loss: 0.6025 - val_accuracy: 0.8443
Epoch 3/200
278/278 [==============================] - 4s 15ms/step - loss: 0.6376 - accuracy: 0.8316 - val_loss: 0.4446 - val_accuracy: 0.8889
Epoch 4/200
278/278 [==============================] - 4s 14ms/step - loss: 0.5099 - accuracy: 0.8675 - val_loss: 0.3666 - val_accuracy: 0.9052
Epoch 5/200
278/278 [==============================] - 4s 14ms/step - loss: 0.4306 - accuracy: 0.8905 - val_loss: 0.3165 - val_accuracy: 0.9200
Epoch 6/200
278/278 [==============================] - 4s 14ms/step - loss: 0.3795 - accuracy: 0.9024 - val_loss: 0.2932 - val_accuracy: 0.9259
Epoch 7/200
278/278 [==============================] - 4s 16ms/step - loss: 0.3403 - accuracy: 0.9120 - val_loss: 0.2894 - val_accuracy:

In [39]:
loss, accuracy = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9684
Testing Accuracy: 0.9141
